In [1]:
import pandas as pd

In [2]:
#Reading and Understanding data


In [3]:
mf_df = pd.read_excel('manuf_data.xlsx') 
rt_df = pd.read_excel('retailer_data.xlsx') 

In [4]:
mf_df.head()

,prod_id,Description
0,355,Mexifiz Shampoo Intensive Repair 400ml
1,356,Mexifiz Shampoo Intensive Repair 400ml
2,381,Cbv Shampoo Moisture Rich 900ml
3,382,Cbv Shampoo Moisture Rich 900ml
4,426,Cbvery Hairspray Extra Hold 400ml


In [5]:
rt_df.head()

,ret_prod_id,Product_Description
0,188,Oddaced Give Me Moisture Shampoo 250ml
1,334,Cbvery Freeze Hold Hairspray 400ml
2,328,Cbvery Firm Hold Hairspray 400ml
3,203,Oddaced Ultimate Hold Hairspray 400ml
4,324,Cbvery Extra Hold Hairspray 400ml


In [6]:
mf_df[mf_df['Description'].isna()]

,prod_id,Description


In [7]:
rt_df[rt_df['Product_Description'].isna()]

,ret_prod_id,Product_Description


In [8]:
mf_df.groupby('prod_id').filter(lambda x :len(x)>1)

,prod_id,Description


In [9]:
mf_df.groupby('prod_id').filter(lambda x :len(x)>1)

,prod_id,Description


In [10]:
mf_df.groupby('Description').filter(lambda x :len(x)>1)

,prod_id,Description
0,355,Mexifiz Shampoo Intensive Repair 400ml
1,356,Mexifiz Shampoo Intensive Repair 400ml
2,381,Cbv Shampoo Moisture Rich 900ml
3,382,Cbv Shampoo Moisture Rich 900ml
5,451,Cbv Shampoo Moisture Rich 900ml
...,...,...
349,2308,Cbv Heat Protection Spr Protect 300ml
354,2336,Cbv Inco Leave on Repair+protect 125ml
355,2337,Cbv Inco Leave on Repair+protect 125ml
359,2349,Twicebeaut Cond 500ml Bot Hydrat


In [11]:
rt_df.groupby('ret_prod_id').filter(lambda x :len(x)>1)

,ret_prod_id,Product_Description


In [12]:
rt_df.groupby('Product_Description').filter(lambda x :len(x)>1)

,ret_prod_id,Product_Description


In [13]:
#Creating company columns

In [14]:
n=rt_df['Product_Description'].str.split(' ',n=1,expand=True)

In [15]:
rt_df['company']=n[0]

In [16]:
n=mf_df['Description'].str.split(' ',n=1,expand=True)

In [17]:
mf_df['company']=n[0]

In [18]:
sorted(mf_df['company'].unique())

['6Good',
 'Baby',
 'Cbv',
 'Cbvery',
 'Fiur',
 'Mexifiz',
 'Oddaced',
 'Parrot',
 'Sinoyez',
 'Toz',
 'Twicebeaut']

In [19]:
import numpy as np
mf_df['company'] = np.where(mf_df['company']=='Cbv','Cbvery',mf_df['company'])

In [20]:
companies=sorted(rt_df['company'].unique())

In [21]:
mf_df=mf_df.sort_values('company')
rt_df=rt_df.sort_values('company')


In [22]:
#Matching strigs with fuzzywuzzy

In [23]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\shanthi\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [24]:
def checker(wrong_options,correct_options):
    names_array=[]
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
            names_array.append(wrong_option)
            ratio_array.append('100')
        else:   
            x=process.extractBests(wrong_option,correct_options,scorer=fuzz.token_set_ratio,limit=3)
            names_array.append([str(x[i][0])+':'+str(x[i][1]) for i in range(len(x))])
    return names_array

In [25]:
whole_names=[]
for company in companies:
    strOptions = mf_df.loc[mf_df['company']==company,'Description'].tolist()
    str2Match = rt_df.loc[rt_df['company']==company,'Product_Description'].tolist()
    name_match=checker(str2Match,strOptions)
    whole_names.extend(name_match)

In [26]:

len(rt_df)

389

In [27]:
rt_df['Description']=whole_names


In [28]:
rt_df=rt_df.explode('Description')

In [29]:
n=rt_df['Description'].str.split(':',expand=True)

In [30]:
rt_df['Description']=n[0]
rt_df['Ratio']=n[1]

In [31]:
len(rt_df)

1135

In [32]:
rt_df=rt_df[['ret_prod_id', 'Product_Description', 'Description',
       'Ratio']].merge(mf_df[['prod_id', 'Description']],on=['Description'])

In [33]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [34]:
rt_df =rt_df.sort_values(['ret_prod_id','Ratio'])

In [35]:
rt_df.tail(30)

,ret_prod_id,Product_Description,Description,Ratio,prod_id
452,1474,6Good Skin Quench Sleeping Cream Skin Face,6Good Co Gentle Care Bot 200ml,48,1289
1277,1482,Cbvery Keratin Hair Conditioner,Cbv Inco Leave on Keratin 120ml,52,1778
1278,1482,Cbvery Keratin Hair Conditioner,Cbv Inco Leave on Keratin 120ml,52,1779
1269,1482,Cbvery Keratin Hair Conditioner,Cbv Hs Finishing Spr Keratin 200ml,58,846
1270,1482,Cbvery Keratin Hair Conditioner,Cbv Hs Finishing Spr Keratin 200ml,58,847
1271,1482,Cbvery Keratin Hair Conditioner,Cbv Hs Finishing Spr Keratin 200ml,58,846
1272,1482,Cbvery Keratin Hair Conditioner,Cbv Hs Finishing Spr Keratin 200ml,58,847
14,1493,Cbvery Freeze Hold 24 hour frizz control Hairspray for a professional look 400 ml pack of 6,Cbv Hairspray Freeze Hold 100ml,81,1385
59,1493,Cbvery Freeze Hold 24 hour frizz control Hairspray for a professional look 400 ml pack of 6,Cbv Hairspray Freeze Hold 250ml,81,1441
11,1493,Cbvery Freeze Hold 24 hour frizz control Hairspray for a professional look 400 ml pack of 6,Cbvery Hairspray Freeze Hold 400ml,90,489


In [36]:
len(rt_df)

1379

In [37]:
sum(rt_df['Ratio'].astype(int))/len(rt_df)

62.22697606961567